In [1]:
import requests
import time
import hmac
import hashlib

BASE_URL = "https://mock-api.roostoo.com"
API_KEY = 'wzVdBhwUlDOB4hhpPIKxh0GSlvUnxWQXpoU7t0o69INPArXcyg0poWOa6m8gtayO'
SECRET = 'vcXTcCASuT1qnlG9DPvG7kZfBJTPTvl0IErYG2QbgRYjgwwwmHxzovJVAjfgEJkp'

In [2]:
def generate_signature(params):
    query_string = '&'.join(["{}={}".format(k, params[k])
                             for k in sorted(params.keys())])
    us = SECRET.encode('utf-8')
    m = hmac.new(us, query_string.encode('utf-8'), hashlib.sha256)
    return m.hexdigest()


def get_server_time():
    r = requests.get(
        BASE_URL + "/v3/serverTime",
    )
    print(r.status_code, r.text)
    return r.json()

def get_ticker(pair=None):
    payload = {
        "timestamp": int(time.time()),
    }
    if pair:
        payload["pair"] = pair

    r = requests.get(
        BASE_URL + "/v3/ticker",
        params=payload,
    )
    print(r.status_code, r.text)
    return r.json()


def get_balance():
    payload = {
        "timestamp": int(time.time()) * 1000,
    }

    r = requests.get(
        BASE_URL + "/v3/balance",
        params=payload,
        headers={"RST-API-KEY": API_KEY,
                 "MSG-SIGNATURE": generate_signature(payload)}
    )
    print(r.status_code, r.text)
    return r.json()

def place_order(coin, side, qty, price=None):
    if side==1:
        side='BUY'
    elif side==-1:
        side='SELL'
    payload = {
        "timestamp": int(time.time()) * 1000,
        "pair": coin + "/USD",
        "side": side,
        "quantity": qty,
    }

    if not price:
        payload['type'] = "MARKET"
    else:
        payload['type'] = "LIMIT"
        payload['price'] = price

    r = requests.post(
        BASE_URL + "/v3/place_order",
        data=payload,
        headers={"RST-API-KEY": API_KEY,
                "MSG-SIGNATURE": generate_signature(payload)}
    )
    print(r.status_code, r.text)

def cancel_order():
    payload = {
        "timestamp": int(time.time()) * 1000,
        # "order_id": 77,
        "pair": "BTC/USD",
    }

    r = requests.post(
        BASE_URL + "/v3/cancel_order",
        data=payload,
        headers={"RST-API-KEY": API_KEY,
                 "MSG-SIGNATURE": generate_signature(payload)}
    )
    print(r.status_code, r.text)


def query_order():
    payload = {
        "timestamp": int(time.time())*1000,
        # "order_id": 77,
        # "pair": "DASH/USD",
        # "pending_only": True,
    }

    r = requests.post(
        BASE_URL + "/v3/query_order",
        data=payload,
        headers={"RST-API-KEY": API_KEY,
                 "MSG-SIGNATURE": generate_signature(payload)}
    )
    print(r.status_code, r.text)


def pending_count():
    payload = {
        "timestamp": int(time.time()) * 1000,
    }

    r = requests.get(
        BASE_URL + "/v3/pending_count",
        params=payload,
        headers={"RST-API-KEY": API_KEY,
                 "MSG-SIGNATURE": generate_signature(payload)}
    )
    print(r.status_code, r.text)
    return r.json()


def get_ex_info():
    r = requests.get(
        BASE_URL + "/v3/exchangeInfo",
    )
    print(r.status_code, r.text)
    return r.json()


In [ ]:
{key: value for key, value in get_balance()['SpotWallet'].it}

200 {"Success":true,"ErrMsg":"","SpotWallet":{"BNB":{"Free":8.06,"Lock":0},"BTC":{"Free":0,"Lock":0},"ETH":{"Free":0,"Lock":0},"USD":{"Free":41237.15,"Lock":0}},"MarginWallet":{}}



ValueError: too many values to unpack (expected 2)

In [12]:
get_balance()['SpotWallet']

200 {"Success":true,"ErrMsg":"","SpotWallet":{"BNB":{"Free":8.06,"Lock":0},"BTC":{"Free":0,"Lock":0},"ETH":{"Free":0,"Lock":0},"USD":{"Free":41237.15,"Lock":0}},"MarginWallet":{}}



{'BNB': {'Free': 8.06, 'Lock': 0},
 'BTC': {'Free': 0, 'Lock': 0},
 'ETH': {'Free': 0, 'Lock': 0},
 'USD': {'Free': 41237.15, 'Lock': 0}}

In [3]:
from exe import Execution

execution = Execution({'BTC': 0.5, 'ETH': 0.5})
execution.send_order({'BTC': 0.3, 'ETH': 0.0})

SSLError: HTTPSConnectionPool(host='mock-api.roostoo.com', port=443): Max retries exceeded with url: /v3/balance?timestamp=1761980145000 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1032)')))

In [36]:
# 1. Get balance
balance_resp = get_balance()
wallet = balance_resp.get("SpotWallet", {})  # or "SpotWallet" if needed

total_value = 0.0

# 2. Loop through each asset in wallet
for asset, balances in wallet.items():
    free_amount = balances.get("Free", 0.0)

    if asset == "USD":
        # USD value is just the amount
        usd_value = free_amount
        # print(f"USD: {free_amount} → ${usd_value:,.2f}")
        total_value += usd_value
    else:
        # For crypto assets: get price via ticker
        try:
            pair = f"{asset}/USD"
            ticker_resp = get_ticker(pair=pair)
            
            # Parse price from response
            # Assume response is like: {"price": "60234.56", ...}
            # or {"data": {"price": "60234.56"}}
            price = None
            price = float(ticker_resp["Data"][pair]["LastPrice"])

            usd_value = free_amount * price
            total_value += usd_value

        except Exception as e:
            print(f"❌ Error fetching price for {asset}: {e}")
            continue


200 {"Success":true,"ErrMsg":"","SpotWallet":{"BNB":{"Free":8.06,"Lock":0},"BTC":{"Free":0.0453,"Lock":0},"ETH":{"Free":4.2736,"Lock":0},"USD":{"Free":19714.02,"Lock":0}},"MarginWallet":{}}

200 {"Success":true,"ErrMsg":"","ServerTime":1761969811969,"Data":{"BNB/USD":{"MaxBid":1090.86,"MinAsk":1090.87,"LastPrice":1090.86,"Change":0.0045,"CoinTradeValue":530579.501,"UnitTradeValue":577691302.10229}}}

200 {"Success":true,"ErrMsg":"","ServerTime":1761969812195,"Data":{"BTC/USD":{"MaxBid":110347.69,"MinAsk":110347.7,"LastPrice":110347.7,"Change":0.0111,"CoinTradeValue":18682.31097,"UnitTradeValue":2052634781.892319}}}

200 {"Success":true,"ErrMsg":"","ServerTime":1761969812339,"Data":{"ETH/USD":{"MaxBid":3863.31,"MinAsk":3863.32,"LastPrice":3863.32,"Change":0.0093,"CoinTradeValue":442626.748,"UnitTradeValue":1705946179.626089}}}



In [78]:
from decimal import Decimal, ROUND_DOWN
import numpy as np

target = {'BTC': 0.2, 'ETH': 1}

target_usd = {}
total_value=total_value
for asset, ratio in {'BTC': 0.5, 'ETH': 0.5}.items():
    target_usd[asset] = total_value * ratio

r=get_balance()
cash=r['SpotWallet']['USD']['Free']

all_assets = set(r['SpotWallet'].keys())

for asset in all_assets:
    if asset == "USD":
        continue  # skip USD
    curr = r['SpotWallet'].get(asset, 0)['Free']
    pair = f"{asset}/USD"
    ticker_resp = get_ticker(pair=pair)
    
    # Parse price from response
    # Assume response is like: {"price": "60234.56", ...}
    # or {"data": {"price": "60234.56"}}
    price = None
    price = float(ticker_resp["Data"][pair]["LastPrice"])
    curr=price*curr
    targ = target.get(asset, 0)*target_usd.get(asset,0)
    if abs(targ - curr) > 100: # can change the minimum threshold here
        delta = np.sign(targ - curr) * max(abs(targ - curr)/price,1/price) # what if negative number?
        delta = Decimal(delta)
        # Round down to 2 decimals → 123.45
        rounded = delta.quantize(Decimal('0.0001'), rounding=ROUND_DOWN) # how about when value < 1usd and u round down the qty
        print(rounded)
        place_order(coin=asset,side=np.sign(delta),qty=abs(rounded))
        time.sleep(1)  # To avoid hitting rate limits
    else:
        continue




200 {"Success":true,"ErrMsg":"","SpotWallet":{"BNB":{"Free":0,"Lock":0},"BTC":{"Free":0.0454,"Lock":0},"ETH":{"Free":6.4748,"Lock":0},"USD":{"Free":19784.8,"Lock":0}},"MarginWallet":{}}

200 {"Success":true,"ErrMsg":"","ServerTime":1761971156232,"Data":{"ETH/USD":{"MaxBid":3862.81,"MinAsk":3862.82,"LastPrice":3862.81,"Change":-0.0017,"CoinTradeValue":431067.6855,"UnitTradeValue":1661358647.798626}}}

200 {"Success":true,"ErrMsg":"","ServerTime":1761971156357,"Data":{"BNB/USD":{"MaxBid":1090.15,"MinAsk":1090.16,"LastPrice":1090.15,"Change":-0.0042,"CoinTradeValue":523193.037,"UnitTradeValue":569632560.92998}}}

200 {"Success":true,"ErrMsg":"","ServerTime":1761971156970,"Data":{"BTC/USD":{"MaxBid":110311.99,"MinAsk":110312,"LastPrice":110312,"Change":0.0048,"CoinTradeValue":18516.89928,"UnitTradeValue":2034722892.0756583}}}



In [79]:
get_balance()

200 {"Success":true,"ErrMsg":"","SpotWallet":{"BNB":{"Free":0,"Lock":0},"BTC":{"Free":0.0454,"Lock":0},"ETH":{"Free":6.4748,"Lock":0},"USD":{"Free":19784.8,"Lock":0}},"MarginWallet":{}}



{'Success': True,
 'ErrMsg': '',
 'SpotWallet': {'BNB': {'Free': 0, 'Lock': 0},
  'BTC': {'Free': 0.0454, 'Lock': 0},
  'ETH': {'Free': 6.4748, 'Lock': 0},
  'USD': {'Free': 19784.8, 'Lock': 0}},
 'MarginWallet': {}}